In [2]:
reviews = io.load('preproc_reviews')

2019-06-14 20:26:55,917 - kedro.io.data_catalog - INFO - Loading data from `preproc_reviews` (PickleLocalDataSet)...


In [5]:
test = reviews.head(n = 10)

In [15]:
words = [text.split() for text in test['review_text']]
words[:3]

[['Absolutely',
  'wonderful',
  '-',
  'silky',
  'and',
  'sexy',
  'and',
  'comfortable'],
 ['Love',
  'this',
  'dress!',
  "it's",
  'sooo',
  'pretty.',
  'i',
  'happened',
  'to',
  'find',
  'it',
  'in',
  'a',
  'store,',
  'and',
  "i'm",
  'glad',
  'i',
  'did',
  'bc',
  'i',
  'never',
  'would',
  'have',
  'ordered',
  'it',
  'online',
  'bc',
  "it's",
  'petite.',
  'i',
  'bought',
  'a',
  'petite',
  'and',
  'am',
  '5\'8".',
  'i',
  'love',
  'the',
  'length',
  'on',
  'me-',
  'hits',
  'just',
  'a',
  'little',
  'below',
  'the',
  'knee.',
  'would',
  'definitely',
  'be',
  'a',
  'true',
  'midi',
  'on',
  'someone',
  'who',
  'is',
  'truly',
  'petite.'],
 ['I',
  'had',
  'such',
  'high',
  'hopes',
  'for',
  'this',
  'dress',
  'and',
  'really',
  'wanted',
  'it',
  'to',
  'work',
  'for',
  'me.',
  'i',
  'initially',
  'ordered',
  'the',
  'petite',
  'small',
  '(my',
  'usual',
  'size)',
  'but',
  'i',
  'found',
  'this',
  'to